In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv('./dataset/clean_v1.csv')
df_test = pd.read_csv('./dataset/test_set_VU_DM.csv')

In [2]:
X_train = df.loc[:, ~df.columns.isin(['srch_id','target_label'])]
y_train = df.loc[:, df.columns.isin(['target_label'])]

# groups = train_data.groupby('srch_id').size().to_frame('size')['size'].to_numpy()
groups = df['srch_id'].to_numpy()

#We need to keep the id for later predictions
X_test = df_test.loc[:, ~df_test.columns.isin(['target_label'])]
y_test = df_test.loc[:, df_test.columns.isin(['target_label'])]

       

In [3]:
import xgboost as xgb
params = {
    'tree_method': 'hist',
    'objective': 'rank:ndcg',  # Use rank:ndcg for optimizing NDCG
    'eval_metric': 'ndcg@5',    # Evaluation metric NDCG@k
    'eta': 0.1,                  # Learning rate
    'max_depth': 6               # Maximum depth of a tree
}

# Initialize the XGBRanker with modified parameters
model = xgb.XGBRanker(**params)

X_train = X_train[['prop_starrating', 'prop_review_score', 'prop_brand_bool', 'prop_location_score1', 'srch_destination_id', 'srch_length_of_stay', 'srch_booking_window',
       'srch_adults_count', 'srch_children_count', 'srch_room_count']]
y_train = y_train['target_label']
model.fit(X_train.to_numpy(), y_train.to_numpy(), qid = groups)

XGBRanker(base_score=None, booster=None, callbacks=None, colsample_bylevel=None,
          colsample_bynode=None, colsample_bytree=None, device=None,
          early_stopping_rounds=None, enable_categorical=False, eta=0.1,
          eval_metric='ndcg@5', feature_types=None, gamma=None,
          grow_policy=None, importance_type=None, interaction_constraints=None,
          learning_rate=None, max_bin=None, max_cat_threshold=None,
          max_cat_to_onehot=None, max_delta_step=None, max_depth=6,
          max_leaves=None, min_child_weight=None, missing=nan,
          monotone_constraints=None, multi_strategy=None, n_estimators=None,
          n_jobs=None, num_parallel_tree=None, ...)

In [4]:
def predict(model, df):
    return model.predict(df.loc[:, ~df.columns.isin(['srch_id'])])

predictions = (X_test[['prop_starrating', 'prop_review_score', 'prop_brand_bool', 'prop_location_score1', 'srch_destination_id', 'srch_length_of_stay', 'srch_booking_window',
       'srch_adults_count', 'srch_children_count', 'srch_room_count', 'srch_id']].groupby('srch_id')
               .apply(lambda x: predict(model, x)))

/var/folders/2p/lk_7jyfx52g86tz59mc412p80000gn/T/ipykernel_64748/2952709984.py:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  predictions = (X_test[['prop_starrating', 'prop_review_score', 'prop_brand_bool', 'prop_location_score1', 'srch_destination_id', 'srch_length_of_stay', 'srch_booking_window',


In [5]:
grouped = X_test.groupby('srch_id')
modified_groups = []

for group_name, group_data in grouped:
    group_data['predicted_reg'] = predictions[group_name]
    modified_groups.append(group_data.copy())
modified_df = pd.concat(modified_groups)

In [6]:
modified_df

,srch_id,date_time,site_id,visitor_location_country_id,visitor_hist_starrating,visitor_hist_adr_usd,prop_country_id,prop_id,prop_starrating,prop_review_score,...,comp6_rate,comp6_inv,comp6_rate_percent_diff,comp7_rate,comp7_inv,comp7_rate_percent_diff,comp8_rate,comp8_inv,comp8_rate_percent_diff,predicted_reg
0,1,2013-02-02 15:27:40,24,216,NaN,NaN,219,3180,3,4.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.279269
1,1,2013-02-02 15:27:40,24,216,NaN,NaN,219,5543,3,4.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.275957
2,1,2013-02-02 15:27:40,24,216,NaN,NaN,219,14142,2,3.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.262973
3,1,2013-02-02 15:27:40,24,216,NaN,NaN,219,22393,3,4.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.275957
4,1,2013-02-02 15:27:40,24,216,NaN,NaN,219,24194,3,4.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.279269
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4959178,332787,2013-05-21 11:06:37,24,216,NaN,NaN,117,32019,4,3.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.282280
4959179,332787,2013-05-21 11:06:37,24,216,NaN,NaN,117,33959,4,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.285190
4959180,332787,2013-05-21 11:06:37,24,216,NaN,NaN,117,35240,4,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.201748
4959181,332787,2013-05-21 11:06:37,24,216,NaN,NaN,117,94437,4,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.214772


In [10]:
result = modified_df.sort_values(by=['srch_id', 'predicted_reg'], ascending=[True, False])
result[['srch_id', 'prop_id']].reset_index(drop=True).to_csv("dataset/submission3_basic_L2R.csv", index=False)